## Blue scheme flow

> JupyterNotebook to visualize `QGIS2OPENDSS` plug-in's logic flow by an abstraction of its methods and functios.

Get an arbitrary layer just as example:

In [1]:
linesData = {
    "ICEobjectID": ['401129__0', '401129__1', '401131__0', '401131__1', '401131__2'],
    "LibraryName": ['LC::BT_6AAAC_AAAC_123.3_6', 'LC::BT_6AAAC_AAAC_123.3_6',
                    'LC::BT_123.3AAAC_AAAC_123.3_1', 'LG::BT_123.3AAAC_AAAC_123.3_1',
                    'BT_123.3AAAC_AAAC_123.3_1'],
    "NEUTMAT": ['AAAC', 'AAAC', 'AAAC', 'AAAC', 'AAAC'],
    "NEUTSIZ": ['123.3', '123.3', '123.3', '123.3', '123.3'],
    "PHASEMAT": ['AAAC', 'AAAC', 'AAAC', 'AAAC', 'AAAC'],
    "PHASESIZ": ['6', '6', '123.3', '123.3', '123.3'],
    "NOMVOLT": [30, 30, 30, 30, 30],
    "TYPE": ['SLC', 'SLC', 'LVC', 'LVC', 'LVC'],
    "LENGTH": [0.004479549358021148, 0.04863058437331125, 0.06930650009633353, 0.0, 0.06228217256034441],
    "X1": [429663.8184, 429665.7786, 430055.120986, 430095.793151, 430095.793151],
    "Y1": [1100682.5245, 1100678.4966, 1100948.733, 1101004.85043, 1101004.85043],
    "X2": [429665.7786, 429635.6291, 430095.793151, 430095.793151, 430132.832372],
    "Y2": [1100678.4966, 1100640.3398, 1101004.85043, 1101004.85043, 1101054.92203]
}


Changes in `qgis2opendss.py` plug-in:

In [2]:

# BLUE modification
# -----------------
nom_volt = linesData['NOMVOLT']       # Line 497
# New lines in Blue code
ICEobjID = linesData["ICEobjectID"]   # New: list[str]
libName = linesData["LibraryName"]    # New: list[str]
# Add to dict as in line 516
UGlineData = {
    "ICEobjectID": ICEobjID,
    "LibraryName": libName
}
# Add to another dict as in line 526
OHlineData = {
    "ICEobjectID": ICEobjID,
    "LibraryName": libName
}

# ... -> Send to Graph object
edges = []
Nlines = len(linesData["ICEobjectID"])
for n in range(Nlines):
    line = {}
    for k in linesData.keys():
        line[k] = linesData[k][n]
        # Other attributes of edge
        line["from_bus"] = "from_busA"
        line["to_bus"] = "to_busB"
    edges.append(line)

In [3]:
# ... -> Line 8443: Get from graph and write *.dss file
for DATA in edges:
    # New lines in Blue code
    libcode = DATA["LibraryName"]

    # New: line 8458
    if "::" in libcode:
        equipment =  libcode.split("::")[1]
    else:
        # About equipment: original lines from 8452 to 8457 (inclusive)
        equipment = DATA["PHASEMAT"] + DATA["PHASESIZ"] + DATA["NEUTSIZ"]

    frombus = DATA["from_bus"]
    tobus = DATA["to_bus"]
    lineLen = "{0:.4f}".format(DATA["LENGTH"])

    if "::" in libcode:
        lineName = DATA["ICEobjectID"]
    else:
        # About lineName: original lines from 8463 to 8466 (inclusive)
        lineName = "MV" + DATA["TYPE"] + 'P' + DATA["NEUTMAT"]

    # Comment "geometry" in line: 8619
    line = 'new line.' + lineName + ' bus1=' + frombus
    line += ' bus2=' +  tobus  # + ' geometry=' + equipment
    # New
    if "LC::" in libcode:
        line += ' linecode=' + equipment
    elif "LG::" in libcode:
        line += ' geometry=' + equipment
    elif "::" not in libcode:
        line += ' geometry=' + equipment
    else:
        raise Exception(("LibraryName can only be LC:: or LG:: "),
                        ("see documentation of Line object of shape.py module."))
    # Continue in line 8620
    line += ' length=' + lineLen + ' units=m'
    print(line)

new line.401129__0 bus1=from_busA bus2=to_busB linecode=BT_6AAAC_AAAC_123.3_6 length=0.0045 units=m
new line.401129__1 bus1=from_busA bus2=to_busB linecode=BT_6AAAC_AAAC_123.3_6 length=0.0486 units=m
new line.401131__0 bus1=from_busA bus2=to_busB linecode=BT_123.3AAAC_AAAC_123.3_1 length=0.0693 units=m
new line.401131__1 bus1=from_busA bus2=to_busB geometry=BT_123.3AAAC_AAAC_123.3_1 length=0.0000 units=m
new line.MVLVCPAAAC bus1=from_busA bus2=to_busB geometry=AAAC123.3123.3 length=0.0623 units=m


Special transformer connections: `Open wye - Open Delta`


```shell

new transformer.T1 bus1=B1.1.0 bus2=B2.1.2 ...
new transformer.T2 bus1=B1.2.0 bus2=B2.2.3 ...
```


### Changes

- Add another conditional in line 790 <br>
- Line 825 change: 'LOADCONNS': '.1.2.3', 'LOADCONF': 'delta' for 'LOADCONNS': phase, 'LOADCONF': 'wye'
Assuming same phase designation of transformer for load service and `wye` for OpenDSS convention in such cases. <br>
- Line 9237 add conditional of Open Delta in secondary, 9239 `OD` True<br>
- Line 9304 msg += 'en el secundario sólo puede'   #  ¿ Sería en el primario? <br>

In [3]:
phase = ".1.3"
ph = "0" + phase
ph = ph.split(".")
primConnA = f".{ph[1]}.{ph[0]}"
primConnB = f".{ph[2]}.{ph[0]}"


In [4]:
# Keep secondary lagging 30deg
if phase == ".1.2":
    secConnA = ".1.2"
    secConnB = ".2.3"
elif phase == ".1.3":
    secConnA = ".1.2"
    secConnB = ".2.3"
elif phase == ".2.3":
    secConnA = ".2.3"
    secConnB = ".3.1"


print(secConnA)
print(secConnB)

.1.2
.2.3


In [6]:
trafName1 = "unit_A"
imagA = "%imag=1.00"
impedanceA = "Xhl=5.90 %Rs=[1.70 1.70]"
noloadlossA = "%noloadloss=0.37"
busMV = "sourcebus"
busLV = "B"
kV_MedLN = "19.92"
kV_LowLL = "0.24"
kVA_trafoA = "50"
tap = "0.95"
normhkva_A = " normhkva=" + kVA_trafoA
grupo_trafo_mv = "209"
trafName2 = "unit_B"
grupo_trafo_mv = "210"
grupo_trafo_lv = "210"

# Write file
line_A = 'new transformer.' + trafName1 + ' phases=1 windings=2 ' + imagA + ' '
line_A += impedanceA + ' ' + noloadlossA + ' buses=[' + busMV + primConnA + ' '
line_A +=  busLV + secConnA + ']'
line_A += ' kvs=[' + kV_MedLN + ' ' + kV_LowLL + '] kvas=[' + kVA_trafoA + ' ' + kVA_trafoA + ']'
line_A += ' conns=[wye wye] Taps=[' + tap + ', 1]' + normhkva_A + ' !GroupMV=' + grupo_trafo_mv
line_B = 'new transformer.' + trafName2 + ' phases=1 windings=2 ' + imagA
line_B += ' ' + impedanceA + ' ' + noloadlossA + ' buses=[' + busMV + primConnB
line_B += ' ' + busLV + secConnB 
line_B += '] kvs=[' + kV_MedLN + ' ' + kV_LowLL + '] kvas=[' + kVA_trafoA + ' '
line_B += kVA_trafoA + '] conns=[wye wye] Taps=[' + tap + ', 1]'
line_B +=  normhkva_A + ' !GroupMV=' + grupo_trafo_mv 
line_A += ' !GroupLV=' + grupo_trafo_lv
line_B += ' !GroupLV=' + grupo_trafo_lv  

print(line_A)
print(line_B)


new transformer.unit_A phases=1 windings=2 %imag=1.00 Xhl=5.90 %Rs=[1.70 1.70] %noloadloss=0.37 buses=[sourcebus.1.0 B.1.2] kvs=[19.92 0.24] kvas=[50 50] conns=[wye wye] Taps=[0.95, 1] normhkva=50 !GroupMV=210 !GroupLV=210
new transformer.unit_B phases=1 windings=2 %imag=1.00 Xhl=5.90 %Rs=[1.70 1.70] %noloadloss=0.37 buses=[sourcebus.3.0 B.2.3] kvs=[19.92 0.24] kvas=[50 50] conns=[wye wye] Taps=[0.95, 1] normhkva=50 !GroupMV=210 !GroupLV=210


In [ ]:
# Define connections primary: OYn
ph = "0" + phase
ph = ph.split(".")
primConnA = f".{ph[1]}.{ph[0]}"
primConnB = f".{ph[2]}.{ph[0]}"
# Define connections secondary: Od
if phase == ".1.2":
    secConnA = ".1.2"
    secConnB = ".2.3"
elif phase == ".1.3":
    secConnA = ".1.2"
    secConnB = ".2.3"
elif phase == ".2.3":
    secConnA = ".2.3"
    secConnB = ".3.1"


In [ ]:
line_A = 'new transformer.' + trafName1 + ' phases=1 windings=2 ' + imagA + ' '
line_A += impedanceA + ' ' + noloadlossA + ' buses=[' + busMV + conexME_trafoA + ' '
line_A +=  busLV + conexBA_trafoA + '] '
line_A += ' kvs=[' + kV_MedLN + ' ' + kV_LowLL + '] kvas=[' + kVA_trafoA + ' ' + kVA_trafoA + ']'
line_A += ' conns=[wye wye] Taps=[' + tap + ', 1]' + normhkva_A + ' !GroupMV=' + grupo_trafo_mv